<a href="https://colab.research.google.com/github/tlchampion/simpsons_character_recognition/blob/main/notebooks/model_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose

Model 3 will use the hyperband method to perform hyperparameter tuning on the CNN model.

<br>


# Setup Environment

## Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install and import modules

In [2]:
! pip install python-dotenv
%pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 14.8 MB/s 


In [3]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import PIL 
import PIL.Image
import pathlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
sys.path.insert(0,"/content/drive/My Drive/Projects/simpsons_character_recognition/simpsons_images")
from dotenv import find_dotenv, load_dotenv
import datetime
import pytz
import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score

In [4]:
from d01_data import make_dataset

In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential
from tensorflow.keras.regularizers import l2

# Copy image files to drive local to colab instance





In [ ]:
%cp -r /content/drive/MyDrive/Projects/simpsons_character_recognition/data /content/data

# Import Data

In [ ]:
train_directory = os.path.join(os.getcwd(),'data/images/simpsons_dataset')
test_directory = os.path.join(os.getcwd(),'data/images/simpsons_testset')
test_labels = os.path.join(os.getcwd(),'data/images/test_labels.csv')


In [ ]:
datagen, datagen_test, train, validation, test = make_dataset.create_datasets(train_directory, test_directory, test_labels)

# Setup Model

In [ ]:
%load_ext tensorboard

In [ ]:


early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=5, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

checkpoint_filepath = '/content/drive/MyDrive/Projects/simpsons_character_recognition/checkpoints/model_3/'
checkpoints = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, monitor='val_loss', verbose=0, save_best_only=False,
    save_weights_only=True, mode='auto', save_freq='epoch',
    options=None, initial_value_threshold=None
)


log_dir = '/content/drive/MyDrive/Projects/simpsons_character_recognition/logdir/' + 'model_3/' 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


LR_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, verbose=0,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0
)

In [ ]:
def build_model(hp):
    units1 = hp.Int('units1', min_value=16, max_value=192, step=16)
    units2 = hp.Int('units2', min_value=16, max_value=192, step=16)
    units3 = hp.Int('units3', min_value=16, max_value=192, step=16)
    lr = hp.Choice('learning rate', values=[1e-2, 1e-3, 1e-4])

    model = keras.Sequential()
    model.add(keras.layers.Conv2d(units1, 3, activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Conv2d(units2, 3, activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Conv2d(units3, 3, activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(42, activation='softmax'))



    f1_metric = tfa.metrics.F1Score(num_classes=42, average='weighted')




    # Compile the model
    model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=lr),
                metrics=['accuracy', 'AUC', f1_metric])

    return model

In [ ]:
tuner = kt.Hyperband(build_model,
                      objective='val_accuracy',
                      max_epochs=200,
                      factor=3,
                      directory="/content/drive/MyDrive/Projects/simpsons_character_recognition/hp"),
                      project_name='model3')

In [ ]:
tuner.search(train, 
              validation_data=validation,
              epochs=200,
              callbacks=[tensorboard_callback,early_stop] )

In [ ]:
%tensorboard --logdir '/content/drive/MyDrive/Projects/simpsons_character_recognition/logdir/'


In [ ]:
tuner.results_summary()

In [ ]:
tuner.get_best_hyperparameters()[0]